# <span style="color:#F26835; font-size:34px;" >Logistic Regression</span>
###### <span style="color:#A1A1A1;"> Students: </span>
###### <span style="color:#A1A1A1;">Instructor: Magnús Eðvald Björnsson </span>

## Question
> ###### <span style="color:#555;">What is Regression ? </span>


Regression is a measure of the relation between the mean value of one variable (e.g., output) and corresponding values of other variables ( time and cost for example).

## Question
> ###### <span style="color:#555;">What is Logistic Regression ? </span>




$ m = \dfrac{ \overline{xy} - \bar{x} \bar{y}} {x_2 - x_1} $

or psuedo-code
```
( meanof(x*y) - (mean_of(x)*mean_of(y)) ) / (mean_of(x^2) - mean_of(x)^2 )

```


## Question 
> ###### <span style="color:#555;"> How is it calculated ? </span>


### <span style="color:#498BA6;">Answer </span>
##### Arranging the formula


$ \bar{x} \text{ and  } \bar{y}_1 \text{ blabla  }  $

#### <span style="color:#F2B33D;">Verifying Results:</span>

Asserting that blabla

### <span style="color:#B19B7D;"> A simple example calculated </span>

## Question
> ###### <span style="color:#555;">Why do we need Logistic Regression ? </span>
